In [1]:
import torch
import torch.nn.functional as F  
from torch import optim 
from torch import nn
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
import torch.nn.init as init
import random
import time

### Class for the Neural Network

In [2]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes, hidden_layer_sizes, activation_function, apply_softmax = False):
        super(NN, self).__init__()

        seed = 18
        torch.manual_seed(seed)

        self.activation = activation_function
        self.apply_softmax = apply_softmax

        # Input layer
        self.input_layer = nn.Linear(input_size, hidden_layer_sizes[0])
        init.kaiming_uniform_(self.input_layer.weight, mode='fan_in', nonlinearity=activation_function.__name__)


        # Hidden layers
        self.hidden_layers = nn.ModuleList([
            nn.Linear(hidden_layer_sizes[i], hidden_layer_sizes[i + 1])
            for i in range(len(hidden_layer_sizes) - 1)
        ])

        for layer in self.hidden_layers:
            init.kaiming_uniform_(layer.weight, mode='fan_in', nonlinearity=activation_function.__name__)


        # Output layer
        self.output_layer = nn.Linear(hidden_layer_sizes[-1], num_classes)
        init.kaiming_uniform_(self.output_layer.weight, mode='fan_in', nonlinearity=activation_function.__name__)


    def forward(self, x):
        x = x.float()
        x = self.activation(self.input_layer(x))

        # Process through hidden layers
        for layer in self.hidden_layers:
            x = self.activation(layer(x))

        if self.apply_softmax:
            x = F.softmax(self.output_layer(x), dim=1)
        else:
            x = self.output_layer(x)

        return x
    

### Function to train the model

In [3]:
def train_model(model, train_loader, optimizer, criterion, num_epochs):
    for epoch in range(num_epochs):
        total_loss = 0.0  # Initialize total loss for the epoch
        num_batches = len(train_loader)

        for batch_idx, (data, targets) in enumerate(train_loader):
            data = data.reshape(data.shape[0], -1)

            scores = model(data)
            loss = criterion(scores, targets)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()  # Accumulate the batch loss

        average_loss = total_loss / num_batches
        print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {average_loss}")
    

### Function to calculate the accuracy

In [4]:
def check_accuracy(loader, model):

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:

            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)

            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples


### Function to split the data into train and test

In [5]:
def train_test_split(data: pd.DataFrame, target_label : str, test_size=0.2, return_torch=None):
        
    # split the data into train and test
    train = data.sample(frac=(1-test_size),random_state=200)
    test = data.drop(train.index)
    
    # split the train and test into X and Y
    train_X = train.drop([target_label], axis=1).values
    train_Y = train[target_label].values
    test_X = test.drop([target_label], axis=1).values
    test_Y = test[target_label].values
    
    if return_torch:
        train_X = torch.tensor(train_X)
        train_Y = torch.tensor(train_Y)
        test_X = torch.tensor(test_X)
        test_Y = torch.tensor(test_Y)
    
    return train_X, train_Y, test_X, test_Y

## Function for Random Search

In [6]:
def random_search(num_iterations, initial_configuration, param_ranges, train_loader, test_loader):
    best_accuracy = 0.0
    best_combination = initial_configuration
    train_times = []
    results = []

    for _ in range(num_iterations):
        # Randomly sample hyperparameter values inthe specified ranges
        sampled_configuration = {
            'Hidden Layer Sizes': [random.randint(param_ranges['min_hidden'], param_ranges['max_hidden']) for _ in range(random.randint(param_ranges['min_layers'], param_ranges['max_layers']))],
            'Activation Function': random.choice(param_ranges['activation_functions']),
            'Learning Rate': random.uniform(param_ranges['min_lr'], param_ranges['max_lr']),
            'Batch Size': random.choice(param_ranges['batch_sizes']),
            'Number of Epochs': random.choice(param_ranges['num_epochs'])
        }

        train_start_time = time.time()
        model = NN(input_size=train_loader.dataset.tensors[0].shape[1],
                   num_classes=num_classes,
                   hidden_layer_sizes=sampled_configuration['Hidden Layer Sizes'],
                   activation_function=sampled_configuration['Activation Function'])
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=sampled_configuration['Learning Rate'])

        train_model(model, train_loader, optimizer, criterion, sampled_configuration['Number of Epochs'])

        train_end_time = time.time()
        train_times.append(train_end_time - train_start_time)
        accuracy_test = check_accuracy(test_loader, model)

        result_entry = {
            'Hidden Layer Sizes': sampled_configuration['Hidden Layer Sizes'],
            'Activation Function': sampled_configuration['Activation Function'].__name__,
            'Learning Rate': sampled_configuration['Learning Rate'],
            'Batch Size': sampled_configuration['Batch Size'],
            'Number of Epochs': sampled_configuration['Number of Epochs'],
            'Accuracy': accuracy_test.item(),
            'Training Time': train_end_time - train_start_time
        }

        results.append(result_entry)

        if accuracy_test > best_accuracy:
            best_accuracy = accuracy_test
            best_combination = sampled_configuration

    results_df = pd.DataFrame(results)
    return best_combination, best_accuracy, results_df

### Random Search that iteratively improves current configuration

In [28]:
def iterative_random_search(num_iterations, initial_configuration, param_ranges, train_loader, test_loader, max_attempts=10):
    best_accuracy = 0.0
    best_combination = initial_configuration
    train_times = []
    results = []

    iteration = 0
    while iteration < num_iterations:
        attempt = 0
        found_improvement = False

        while attempt < max_attempts:
            # Randomly sample hyperparameter values in the specified ranges
            sampled_configuration = {
                'Hidden Layer Sizes': [random.randint(param_ranges['min_hidden'], param_ranges['max_hidden']) for _ in range(random.randint(param_ranges['min_layers'], param_ranges['max_layers']))],
                'Activation Function': random.choice(param_ranges['activation_functions']),
                'Learning Rate': random.uniform(param_ranges['min_lr'], param_ranges['max_lr']),
                'Batch Size': random.choice(param_ranges['batch_sizes']),
                'Number of Epochs': random.choice(param_ranges['num_epochs'])
            }

            train_start_time = time.time()
            model = NN(input_size=train_loader.dataset.tensors[0].shape[1],
                       num_classes=num_classes,
                       hidden_layer_sizes=sampled_configuration['Hidden Layer Sizes'],
                       activation_function=sampled_configuration['Activation Function'])
            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=sampled_configuration['Learning Rate'])

            train_model(model, train_loader, optimizer, criterion, sampled_configuration['Number of Epochs'])

            train_end_time = time.time()
            train_times.append(train_end_time - train_start_time)
            accuracy_test = check_accuracy(test_loader, model)

            result_entry = {
                'Hidden Layer Sizes': sampled_configuration['Hidden Layer Sizes'],
                'Activation Function': sampled_configuration['Activation Function'].__name__,
                'Learning Rate': sampled_configuration['Learning Rate'],
                'Batch Size': sampled_configuration['Batch Size'],
                'Number of Epochs': sampled_configuration['Number of Epochs'],
                'Accuracy': accuracy_test.item(),
                'Training Time': train_end_time - train_start_time
            }

            results.append(result_entry)

            # Check if the accuracy of the current configuration is better than the best so far
            if accuracy_test > best_accuracy:
                best_accuracy = accuracy_test
                best_combination = sampled_configuration
                found_improvement = True
                break  # Exit the attempt loop if an improvement is found

            attempt += 1

        # Increment iteration only if an improvement is found or max attempts reached
        iteration += 1 if found_improvement else 0

    results_df = pd.DataFrame(results)
    return best_combination, best_accuracy, results_df

## Function for Local Search

In [7]:
def local_search(num_iterations, initial_configuration, param_ranges, train_loader, test_loader):
    best_accuracy = 0.0
    best_combination = None
    current_configuration = initial_configuration
    train_times = []
    results = []

    for _ in range(num_iterations):
        # small changes to the current configuration
        new_configuration = {
            'Hidden Layer Sizes': [
                max(1, size + random.randint(-1, 1)) for size in current_configuration['Hidden Layer Sizes']
            ],
            'Activation Function': random.choice(param_ranges['activation_functions']),
            'Learning Rate': max(param_ranges['min_lr'], min(param_ranges['max_lr'], current_configuration['Learning Rate'] + random.uniform(-0.01, 0.01))),
            'Batch Size': random.choice(param_ranges['batch_sizes']),
            'Number of Epochs': max(1, current_configuration['Number of Epochs'] + random.randint(-1, 1))
        }

        train_start_time = time.time()
        model = NN(input_size=train_loader.dataset.tensors[0].shape[1],
                   num_classes=num_classes,
                   hidden_layer_sizes=new_configuration['Hidden Layer Sizes'],
                   activation_function=new_configuration['Activation Function'])
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=new_configuration['Learning Rate'])

        train_model(model, train_loader, optimizer, criterion, new_configuration['Number of Epochs'])

        train_end_time = time.time()
        train_times.append(train_end_time - train_start_time)
        accuracy_test = check_accuracy(test_loader, model)

        result_entry = {
            'Hidden Layer Sizes': new_configuration['Hidden Layer Sizes'],
            'Activation Function': new_configuration['Activation Function'].__name__,
            'Learning Rate': new_configuration['Learning Rate'],
            'Batch Size': new_configuration['Batch Size'],
            'Number of Epochs': new_configuration['Number of Epochs'],
            'Accuracy': accuracy_test.item(),
            'Training Time': train_end_time - train_start_time
        }

        results.append(result_entry)

        if accuracy_test > best_accuracy:
            best_accuracy = accuracy_test
            best_combination = new_configuration

        # Update the current configuration for the next iteration
        current_configuration = new_configuration

    results_df = pd.DataFrame(results)
    return best_combination, best_accuracy, results_df

## Loading / preparing data

### Wine Quality

In [8]:
wine_quality = pd.read_csv('./preprocessed-datasets/wine_quality_prepro.csv', index_col=0)
wine_quality.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,class,wine_type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1


### Congressional Voting

In [9]:
cong_voting = pd.read_csv('./preprocessed-datasets/CongressionVoting_prepro.csv')
# encode class value democrat as 1 and republican as 0
cong_voting['class'] = cong_voting['class'].map({'democrat': 1, 'republican': 0})
cong_voting.head()

,ID,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-crporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa,class
0,140,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1
1,383,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1
2,201,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1
3,297,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0
4,309,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0


### Bank Marketing

In [10]:
bank_marketing = pd.read_csv('./preprocessed-datasets/bank_marketing_prepro.csv')
column_to_move = 'class'

# Move class to the last index
columns = [col for col in bank_marketing.columns if col != column_to_move] + [column_to_move]
bank_marketing = bank_marketing[columns]

bank_marketing.drop('Unnamed: 0', axis=1,inplace=True)
bank_marketing.head()

,age,default,housing,loan,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,...,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,poutcome_failure,poutcome_nonexistent,poutcome_success,class
0,56,0.0,0.0,0.0,1,999,0,1.1,93.994,-36.4,...,0,0,0,0,0,0,0,1,0,0
1,57,0.0,0.0,0.0,1,999,0,1.1,93.994,-36.4,...,0,1,0,0,0,0,0,1,0,0
2,37,0.0,1.0,0.0,1,999,0,1.1,93.994,-36.4,...,0,1,0,0,0,0,0,1,0,0
3,40,0.0,0.0,0.0,1,999,0,1.1,93.994,-36.4,...,0,0,0,0,0,0,0,1,0,0
4,56,0.0,0.0,1.0,1,999,0,1.1,93.994,-36.4,...,0,1,0,0,0,0,0,1,0,0


# Test Random Search over all three datasets

In [11]:
datasets = {'wine_quality': wine_quality, 'cong_voting': cong_voting, 'bank_marketing': bank_marketing}
num_iterations = 10
param_ranges = {
    'min_hidden': 5,
    'max_hidden': 50,
    'min_layers': 1,
    'max_layers': 3,
    'activation_functions': [F.relu, F.tanh, F.sigmoid],
    'min_lr': 0.001,
    'max_lr': 0.1,
    'batch_sizes': [32, 64, 128],
    'num_epochs': [5, 10, 15]
}

initial_configuration = {
    'Hidden Layer Sizes': [25],
    'Activation Function': F.relu,
    'Learning Rate': 0.01,
    'Batch Size': 64,
    'Number of Epochs': 10
}



all_random_results = []

for dataset_name, dataset in datasets.items():
    train_X, train_Y, test_X, test_Y = train_test_split(dataset, "class", return_torch=True)

    train_data = TensorDataset(train_X, train_Y)
    train_loader = DataLoader(train_data, batch_size=32, shuffle=False)

    test_data = TensorDataset(test_X, test_Y)
    test_loader = DataLoader(test_data, batch_size=32, shuffle=False)
    
    input_size = train_X.shape[1]

    if dataset_name == 'wine_quality':
        num_classes = 10
    else:
        num_classes = len(np.unique(train_Y))

    best_config, best_accuracy, random_results = random_search(num_iterations, initial_configuration, param_ranges, train_loader, test_loader)

    random_results['dataset'] = dataset_name
    all_random_results.append(random_results)
#all_random_results = pd.DataFrame(all_random_results)
#random_results_df = pd.concat(all_random_results, ignore_index=True)


Epoch 1/10, Average Loss: 1.350421292284515
Epoch 2/10, Average Loss: 1.3346785361781441
Epoch 3/10, Average Loss: 1.3351237682477097
Epoch 4/10, Average Loss: 1.335351613767308
Epoch 5/10, Average Loss: 1.3354768482453985
Epoch 6/10, Average Loss: 1.3355534504527695
Epoch 7/10, Average Loss: 1.3356038119895326
Epoch 8/10, Average Loss: 1.3356388500131713
Epoch 9/10, Average Loss: 1.3356640737480912
Epoch 10/10, Average Loss: 1.3356826115239617
Epoch 1/10, Average Loss: 2.4771564899046727
Epoch 2/10, Average Loss: 1.2859452034066792
Epoch 3/10, Average Loss: 1.2838003050330227
Epoch 4/10, Average Loss: 1.2830607627798443
Epoch 5/10, Average Loss: 1.2826893768427563
Epoch 6/10, Average Loss: 1.2824773532481282
Epoch 7/10, Average Loss: 1.2823466976727445
Epoch 8/10, Average Loss: 1.2822616963298774
Epoch 9/10, Average Loss: 1.2822041584670179
Epoch 10/10, Average Loss: 1.2821640558769367
Epoch 1/15, Average Loss: 1.3094034663007303
Epoch 2/15, Average Loss: 1.2858226211524448
Epoch 3/15

Epoch 1/15, Average Loss: 0.3647539742315839
Epoch 2/15, Average Loss: 0.35762265513479274
Epoch 3/15, Average Loss: 0.3575754541578223
Epoch 4/15, Average Loss: 0.35755982410198855
Epoch 5/15, Average Loss: 0.35755460696900715
Epoch 6/15, Average Loss: 0.3575527845429781
Epoch 7/15, Average Loss: 0.35755212416926635
Epoch 8/15, Average Loss: 0.3575518901283554
Epoch 9/15, Average Loss: 0.3575518041645692
Epoch 10/15, Average Loss: 0.3575517400344201
Epoch 11/15, Average Loss: 0.3575517476224386
Epoch 12/15, Average Loss: 0.3575517522130451
Epoch 13/15, Average Loss: 0.3575517495953957
Epoch 14/15, Average Loss: 0.35755176429401686
Epoch 15/15, Average Loss: 0.3575517506668688
Epoch 1/10, Average Loss: 0.338196681533913
Epoch 2/10, Average Loss: 0.3379244083629071
Epoch 3/10, Average Loss: 0.33793431077586505
Epoch 4/10, Average Loss: 0.3393674987540083
Epoch 5/10, Average Loss: 0.33994367238568163
Epoch 6/10, Average Loss: 0.33971488266025934
Epoch 7/10, Average Loss: 0.33998323065382

In [12]:
random_results_df = pd.DataFrame(all_random_results)
random_results_df = pd.concat(all_random_results, ignore_index=True)

In [13]:
random_results_df

,Hidden Layer Sizes,Activation Function,Learning Rate,Batch Size,Number of Epochs,Accuracy,Training Time,dataset
0,"[47, 21]",tanh,0.044015,128,10,0.284967,3.484986,wine_quality
1,"[36, 30]",relu,0.085153,128,10,0.284967,2.869570,wine_quality
2,"[14, 5]",sigmoid,0.030495,32,15,0.453595,3.610264,wine_quality
3,"[30, 44]",tanh,0.067191,128,10,0.284967,3.481614,wine_quality
4,"[7, 26]",sigmoid,0.020540,128,5,0.471895,2.191039,wine_quality
5,"[12, 24, 21]",sigmoid,0.057690,64,10,0.284967,2.407728,wine_quality
6,[33],sigmoid,0.076724,64,5,0.284967,0.835600,wine_quality
7,[24],sigmoid,0.074965,128,15,0.284967,2.229061,wine_quality
8,"[45, 20, 47]",relu,0.041005,32,5,0.458824,1.026674,wine_quality
9,[8],sigmoid,0.032918,128,5,0.458824,0.684836,wine_quality


### Testing iterative random search

In [ ]:
datasets = {'wine_quality': wine_quality, 'cong_voting': cong_voting, 'bank_marketing': bank_marketing}
num_iterations = 20
param_ranges = {
    'min_hidden': 5,
    'max_hidden': 50,
    'min_layers': 1,
    'max_layers': 3,
    'activation_functions': [F.relu, F.tanh, F.sigmoid],
    'min_lr': 0.001,
    'max_lr': 0.1,
    'batch_sizes': [32, 64, 128],
    'num_epochs': [5, 10, 15]
}

initial_configuration = {
    'Hidden Layer Sizes': [25],
    'Activation Function': F.relu,
    'Learning Rate': 0.01,
    'Batch Size': 64,
    'Number of Epochs': 10
}



all_random_results = []

for dataset_name, dataset in datasets.items():
    train_X, train_Y, test_X, test_Y = train_test_split(dataset, "class", return_torch=True)

    train_data = TensorDataset(train_X, train_Y)
    train_loader = DataLoader(train_data, batch_size=32, shuffle=False)

    test_data = TensorDataset(test_X, test_Y)
    test_loader = DataLoader(test_data, batch_size=32, shuffle=False)
    
    input_size = train_X.shape[1]

    if dataset_name == 'wine_quality':
        num_classes = 10
    else:
        num_classes = len(np.unique(train_Y))

    best_config, best_accuracy, random_results = iterative_random_search(num_iterations, initial_configuration, param_ranges, train_loader, test_loader)

    random_results['dataset'] = dataset_name
    all_random_results.append(random_results)

Epoch 1/10, Average Loss: 2.1231026225294802
Epoch 2/10, Average Loss: 1.2748544157648378
Epoch 3/10, Average Loss: 1.2727871950418672
Epoch 4/10, Average Loss: 1.2628808591994771
Epoch 5/10, Average Loss: 1.255643824858168
Epoch 6/10, Average Loss: 1.249198491222288
Epoch 7/10, Average Loss: 1.2145846197210206
Epoch 8/10, Average Loss: 1.1959362944210965
Epoch 9/10, Average Loss: 1.1940213200504795
Epoch 10/10, Average Loss: 1.205856188674646
Epoch 1/5, Average Loss: 4.182079624544623
Epoch 2/5, Average Loss: 1.2761574190087113
Epoch 3/5, Average Loss: 1.27579409505692
Epoch 4/5, Average Loss: 1.2561285938221984
Epoch 5/5, Average Loss: 1.241655422500306
Epoch 1/15, Average Loss: 2.919040676274914
Epoch 2/15, Average Loss: 1.2821570225288532
Epoch 3/15, Average Loss: 1.2806981077954813
Epoch 4/15, Average Loss: 1.2805765384545356
Epoch 5/15, Average Loss: 1.2805704017358324
Epoch 6/15, Average Loss: 1.2805872226785298
Epoch 7/15, Average Loss: 1.2806079446172423
Epoch 8/15, Average Lo

Epoch 15/15, Average Loss: 1.3807772895309822
Epoch 1/5, Average Loss: 1.3201608910151055
Epoch 2/5, Average Loss: 1.2933789360011283
Epoch 3/5, Average Loss: 1.288962444644764
Epoch 4/5, Average Loss: 1.2872649921230013
Epoch 5/5, Average Loss: 1.2854084610207681
Epoch 1/10, Average Loss: 1.33055215818019
Epoch 2/10, Average Loss: 1.2995264471674257
Epoch 3/10, Average Loss: 1.3020409075029058
Epoch 4/10, Average Loss: 1.3155862495211736
Epoch 5/10, Average Loss: 1.3140117741801256
Epoch 6/10, Average Loss: 1.3129291505170015
Epoch 7/10, Average Loss: 1.314842812122743
Epoch 8/10, Average Loss: 1.3148089888637051
Epoch 9/10, Average Loss: 1.3146342236571518
Epoch 10/10, Average Loss: 1.3134720983680772
Epoch 1/5, Average Loss: 1.4881837163234781
Epoch 2/5, Average Loss: 1.4703255847919208
Epoch 3/5, Average Loss: 1.469740937092553
Epoch 4/5, Average Loss: 1.4684203672994134
Epoch 5/5, Average Loss: 1.4679228988893194
Epoch 1/5, Average Loss: 2.8052529337947356
Epoch 2/5, Average Loss:

Epoch 9/15, Average Loss: 1.2846512224045268
Epoch 10/15, Average Loss: 1.2847407493123248
Epoch 11/15, Average Loss: 1.2846903844845075
Epoch 12/15, Average Loss: 1.284700098944588
Epoch 13/15, Average Loss: 1.2846622028233814
Epoch 14/15, Average Loss: 1.2845110286232884
Epoch 15/15, Average Loss: 1.2831989958242405
Epoch 1/10, Average Loss: 5.419460431198401
Epoch 2/10, Average Loss: 1.2834290002752666
Epoch 3/10, Average Loss: 1.2821463287973696
Epoch 4/10, Average Loss: 1.281914824357062
Epoch 5/10, Average Loss: 1.2818310494803211
Epoch 6/10, Average Loss: 1.2817912599060433
Epoch 7/10, Average Loss: 1.281772476763813
Epoch 8/10, Average Loss: 1.2817651355193436
Epoch 9/10, Average Loss: 1.2817642176809487
Epoch 10/10, Average Loss: 1.2817669178079243
Epoch 1/10, Average Loss: 1.312052135818575
Epoch 2/10, Average Loss: 1.2875613113122484
Epoch 3/10, Average Loss: 1.2854010254327506
Epoch 4/10, Average Loss: 1.2843993690116273
Epoch 5/10, Average Loss: 1.2841379993532334
Epoch 6/

Epoch 2/15, Average Loss: 1.9158207807072833
Epoch 3/15, Average Loss: 1.6365157515724744
Epoch 4/15, Average Loss: 1.5521939195738248
Epoch 5/15, Average Loss: 1.397959615189605
Epoch 6/15, Average Loss: 1.3425945826103352
Epoch 7/15, Average Loss: 1.3069649250960789
Epoch 8/15, Average Loss: 1.2651477532884094
Epoch 9/15, Average Loss: 1.2315534905421954
Epoch 10/15, Average Loss: 1.2386594052695057
Epoch 11/15, Average Loss: 1.206914153567121
Epoch 12/15, Average Loss: 1.1735129612355144
Epoch 13/15, Average Loss: 1.1635035835892145
Epoch 14/15, Average Loss: 1.1479423185067674
Epoch 15/15, Average Loss: 1.1417034736440226
Epoch 1/15, Average Loss: 3.9607868582193104
Epoch 2/15, Average Loss: 1.2736538959427115
Epoch 3/15, Average Loss: 1.2816324511920016
Epoch 4/15, Average Loss: 1.2831827870175883
Epoch 5/15, Average Loss: 1.2830924234507275
Epoch 6/15, Average Loss: 1.2829996822801835
Epoch 7/15, Average Loss: 1.2829232296329334
Epoch 8/15, Average Loss: 1.2828755671261278
Epoch 

Epoch 11/15, Average Loss: 1.2812584170534567
Epoch 12/15, Average Loss: 1.2827648546066752
Epoch 13/15, Average Loss: 1.2814321064510228
Epoch 14/15, Average Loss: 1.2817237267464947
Epoch 15/15, Average Loss: 1.2814160183163508
Epoch 1/15, Average Loss: 1.3598373492071234
Epoch 2/15, Average Loss: 1.3134322137189058
Epoch 3/15, Average Loss: 1.316359936825337
Epoch 4/15, Average Loss: 1.3177098364917779
Epoch 5/15, Average Loss: 1.315910209175999
Epoch 6/15, Average Loss: 1.3167116963790237
Epoch 7/15, Average Loss: 1.3160802235632587
Epoch 8/15, Average Loss: 1.316229602310555
Epoch 9/15, Average Loss: 1.3152509211031205
Epoch 10/15, Average Loss: 1.3167481107945822
Epoch 11/15, Average Loss: 1.3166417402723816
Epoch 12/15, Average Loss: 1.3165245151227236
Epoch 13/15, Average Loss: 1.3164315391903274
Epoch 14/15, Average Loss: 1.3163069991246323
Epoch 15/15, Average Loss: 1.3163495114975912
Epoch 1/15, Average Loss: 1.306605230079838
Epoch 2/15, Average Loss: 1.2720821788705932
Epo

...

Epoch 3/5, Average Loss: 1.3935824522942852
Epoch 4/5, Average Loss: 1.3930137815650987
Epoch 5/5, Average Loss: 1.3946387563015055
Epoch 1/10, Average Loss: 4.331151811988807
Epoch 2/10, Average Loss: 1.2620059880742267
Epoch 3/10, Average Loss: 1.236492927454732
Epoch 4/10, Average Loss: 1.2342654645808635
Epoch 5/10, Average Loss: 1.21783302495816
Epoch 6/10, Average Loss: 1.2289197759394266
Epoch 7/10, Average Loss: 1.2012779204391995
Epoch 8/10, Average Loss: 1.2180683302001718
Epoch 9/10, Average Loss: 1.204477434509371
Epoch 10/10, Average Loss: 1.2067479253546591
Epoch 1/10, Average Loss: 1.315376512843407
Epoch 2/10, Average Loss: 1.3016296515435528
Epoch 3/10, Average Loss: 1.2974206655303393
Epoch 4/10, Average Loss: 1.2955866540136514
Epoch 5/10, Average Loss: 1.2972005854355046
Epoch 6/10, Average Loss: 1.2948473158058214
Epoch 7/10, Average Loss: 1.2907670151236599
Epoch 8/10, Average Loss: 1.2908124397137413
Epoch 9/10, Average Loss: 1.2908920364145853
Epoch 10/10, Avera

Epoch 1/5, Average Loss: 1.6675146211144383
Epoch 2/5, Average Loss: 1.2845239683162946
Epoch 3/5, Average Loss: 1.2829226138401617
Epoch 4/5, Average Loss: 1.2824311051631998
Epoch 5/5, Average Loss: 1.2822243326280747
Epoch 1/15, Average Loss: 1.3374500921898824
Epoch 2/15, Average Loss: 1.3128230791150426
Epoch 3/15, Average Loss: 1.3131721751090208
Epoch 4/15, Average Loss: 1.3133079537584738
Epoch 5/15, Average Loss: 1.313380566842717
Epoch 6/15, Average Loss: 1.3134254363416895
Epoch 7/15, Average Loss: 1.3134555736202405
Epoch 8/15, Average Loss: 1.313476980098186
Epoch 9/15, Average Loss: 1.3134927149930615
Epoch 10/15, Average Loss: 1.313504625683182
Epoch 11/15, Average Loss: 1.313513836977672
Epoch 12/15, Average Loss: 1.313521029027693
Epoch 13/15, Average Loss: 1.3135267269391955
Epoch 14/15, Average Loss: 1.3135312612802705
Epoch 15/15, Average Loss: 1.31353495311152
Epoch 1/10, Average Loss: 1.3373335468257133
Epoch 2/10, Average Loss: 1.3131353467520028
Epoch 3/10, Aver

Epoch 9/10, Average Loss: 1.2808264413494275
Epoch 10/10, Average Loss: 1.2807338011045397
Epoch 1/15, Average Loss: 2.490900317584079
Epoch 2/15, Average Loss: 1.3030116660463298
Epoch 3/15, Average Loss: 1.2884738372147448
Epoch 4/15, Average Loss: 1.2840618092589584
Epoch 5/15, Average Loss: 1.2821840252612997
Epoch 6/15, Average Loss: 1.2812120892518868
Epoch 7/15, Average Loss: 1.2806354885452365
Epoch 8/15, Average Loss: 1.2802594351622225
Epoch 9/15, Average Loss: 1.279997876816732
Epoch 10/15, Average Loss: 1.2798075756412342
Epoch 11/15, Average Loss: 1.2796644696428732
Epoch 12/15, Average Loss: 1.2795539070492141
Epoch 13/15, Average Loss: 1.2794665580878228
Epoch 14/15, Average Loss: 1.2793962012039373
Epoch 15/15, Average Loss: 1.279338642132063
Epoch 1/5, Average Loss: 2.845201190263947
Epoch 2/5, Average Loss: 1.2844049813557257
Epoch 3/5, Average Loss: 1.2828191460275942
Epoch 4/5, Average Loss: 1.2823200160008998
Epoch 5/5, Average Loss: 1.2821011353124139
Epoch 1/10, 

Epoch 3/10, Average Loss: 1.281678957441833
Epoch 4/10, Average Loss: 1.28026465257984
Epoch 5/10, Average Loss: 1.27934548679305
Epoch 6/10, Average Loss: 1.2773750435355251
Epoch 7/10, Average Loss: 1.2777743456553827
Epoch 8/10, Average Loss: 1.2725914142614494
Epoch 9/10, Average Loss: 1.2696505170658323
Epoch 10/10, Average Loss: 1.256350482534046
Epoch 1/5, Average Loss: 1.378816814510369
Epoch 2/5, Average Loss: 1.3508359010965547
Epoch 3/5, Average Loss: 1.3513161068313693
Epoch 4/5, Average Loss: 1.350761007677558
Epoch 5/5, Average Loss: 1.3526639777458518
Epoch 1/15, Average Loss: 1.333090624194935
Epoch 2/15, Average Loss: 1.3066130496241564
Epoch 3/15, Average Loss: 1.3014758518137084
Epoch 4/15, Average Loss: 1.297562689137605
Epoch 5/15, Average Loss: 1.297617533455597
Epoch 6/15, Average Loss: 1.2978680492178793
Epoch 7/15, Average Loss: 1.2980380167990375
Epoch 8/15, Average Loss: 1.2981581073597166
Epoch 9/15, Average Loss: 1.2982448039610692
Epoch 10/15, Average Loss

Epoch 2/15, Average Loss: 1.2890996004174824
Epoch 3/15, Average Loss: 1.285728428992757
Epoch 4/15, Average Loss: 1.2837999971366367
Epoch 5/15, Average Loss: 1.2830090639781366
Epoch 6/15, Average Loss: 1.2826776555710775
Epoch 7/15, Average Loss: 1.2825089868592339
Epoch 8/15, Average Loss: 1.282413144784471
Epoch 9/15, Average Loss: 1.282354631306935
Epoch 10/15, Average Loss: 1.2823169085145727
Epoch 11/15, Average Loss: 1.2822913787116301
Epoch 12/15, Average Loss: 1.2822733949298508
Epoch 13/15, Average Loss: 1.2822602453407335
Epoch 14/15, Average Loss: 1.2822503414622113
Epoch 15/15, Average Loss: 1.2822426345450746
Epoch 1/5, Average Loss: 1.5171954288073113
Epoch 2/5, Average Loss: 1.457629815566759
Epoch 3/5, Average Loss: 1.4573216398069464
Epoch 4/5, Average Loss: 1.4598052187199972
Epoch 5/5, Average Loss: 1.4615999814922824
Epoch 1/15, Average Loss: 4.089726864925923
Epoch 2/15, Average Loss: 1.2859949868149552
Epoch 3/15, Average Loss: 1.2857937022952215
Epoch 4/15, Av